In [ ]:
# 📌 Bu not defterinde yer alan içerikler, ağırlıklı olarak DataCamp platformundaki ilgili eğitim modülleri temel alınarak hazırlanmıştır.
# Ancak içerik yalnızca DataCamp ile sınırlı değildir; Hugging Face dokümantasyonları, PEFT kütüphanesi belgeleri,
# arXiv akademik makaleleri, GitHub açık kaynak projeleri, Stack Overflow, Medium yazıları ve çeşitli teknik bloglar dahil olmak üzere
# birçok farklı platform ve kaynaktan edinilen bilgilerle zenginleştirilmiştir.
#
# Amaç, eğitim sürecimi daha derinlemesine ve çok yönlü bir şekilde desteklemek; konuyu yalnızca tek bir kaynaktan değil,
# farklı bakış açılarıyla öğrenerek kalıcı bir bilgi temeli oluşturmaktır.
#
# Bu içerikler, herhangi bir ticari amaç taşımadan, yalnızca kişisel öğrenim, uygulama ve arşivleme amacıyla kullanılmaktadır.
# Telif hakkı sahiplerine saygı gereği, orijinal kaynaklara doğrudan atıf yapılmaya özen gösterilmiştir. 🙏
#
# Eğitim kapsamında verilen görevleri kendi öğrenim sürecim için yeniden uyguluyorum.
# Orijinal senaryo ve veri yapısı ilgili platforma aittir.

https://campus.datacamp.com/courses/fine-tuning-with-llama-3

In [2]:
! pip install torchtune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.7/910.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
# ! tune ls

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torchtune/__init__.py", line 16, in <module>
    import torchao  # noqa
    ^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'torchao'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/bin/tune", line 5, in <module>
    from torchtune._cli.tune import main
  File "/usr/local/lib/python3.11/dist-packages/torchtune/__init__.py", line 18, in <module>
    raise ImportError(
ImportError: 
        torchao not installed.
        Please follow the instructions at https://pytorch.org/torchtune/main/install.html#pre-requisites
        to install torchao.
        


In [3]:
from datasets import load_dataset, Dataset

In [3]:
ds = load_dataset('bitext/Bitext-customer-support-llm-chatbot-training-dataset', split='train')
# Yüklendikten sonra, veri kümesinin talimat ve yanıt gibi görevimizle alakalı sütunlarını görebiliriz

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)t_Training_Dataset_27K_responses-v11.csv:   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

In [4]:
ds.column_names

['flags', 'instruction', 'category', 'intent', 'response']

In [5]:
import pprint

In [6]:
pprint.pprint(ds[0])

{'category': 'ORDER',
 'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order "
             "{{Order Number}}, and I'm here to provide you with the "
             'information you need. Please go ahead and ask your question, and '
             "I'll do my best to assist you."}


In [7]:
print(ds.shape)

(26872, 5)


In [11]:
first_thousand_points = ds[:1000]
ds = Dataset.from_dict(first_thousand_points)

In [12]:
ds

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 1000
})

In [14]:
def merge_example(row):
  row['conversation'] = f"Query: {row['instruction']}\nResponse: {row['response']}"
  return row

ds = ds.map(merge_example)
print(ds[0]['conversation'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Query: question about cancelling order {{Order Number}}
Response: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [15]:
ds.save_to_disk('preprocessed_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
from datasets import load_from_disk

In [1]:
ds_preprocessed = load_from_disk('preprocessed_dataset')

NameError: name 'load_from_disk' is not defined

In [18]:
ds_preprocessed

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response', 'conversation'],
    num_rows: 1000
})

In [4]:
ds = load_from_disk('preprocessed_dataset')

FileNotFoundError: Directory preprocessed_dataset not found

In [1]:
import yaml

In [ ]:
config_dict = {
    "batch_size": 4,
    "device": "cuda",
    "model" : {
        "_component_": "torchtune.models.llama3_2.llama3_2_1b"
    },
    #...
    }

In [ ]:
yaml_file_path = "custom_recipe.yaml"
with open(yaml_file_path, "w") as yaml_file:
  yaml.dump(config_dict, yaml_file)

In [ ]:
# Konfigürasyonlar da hazır olduğuna göre, fine tune başlatılabilir
# Aşağıdaki komut, custom_recipe.yaml yapılandırma dosyası ile ince ayar işini doğru şekilde çalıştırır
!tune run --config custom_recipe.yaml

In [ ]:
# Model Fine-tunşng
model_name = "Maykeye/TinyLLama-v0"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Bir cümle bittiğinde belirteç üretimini durdurmak için pad_token'ı cümle sonu
# belirtecine ayarlarız
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import TrainingArguments

In [1]:
# HuggingFace'in TrainingArguments yardımcı sınıfı kullanılarak eğitim argümanları yapılandırma:
# https://huggingface.co/docs/transformers/v4.40.1/en/main_classes/trainer#transformers.TrainingArguments
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    learning_rate=2e-3,
    max_grad_norm=0.3,
    max_steps=200,
    #...
    gradient_accumulation_steps=2,
    save_steps=10
  )

In [ ]:
# SFTTrainer'ı kullanmak için, modeli, tokenizer i ve training dataset i
# aşağıdakilerle sağlamak gerekir:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset= dataset,
    dataset_text_field="conversation",
    args=training_arguments,
    max_seq_length=250
)

In [ ]:
# Eğitim başlatılır
trainer.train()

In [2]:
# Model ROUGE1 metriği kullanılarak değerlendirilir.
# https://huggingface.co/spaces/evaluate-metric/rouge
import evaluate

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "master yoda"]

In [ ]:
results = rouge.compute(predictions=predictions, references=references)
print(results)
# İlk örnek tam eşleşme sağlar bu yüzden 1 puan alır, ikinci örnek eşleşme sağlamaz, 0 puan alır.
# Output: {'rouge1': 0.5, 'rouge2': 0,5, 'rougeL': 0.5, 'rougeLsum': 0.5}

In [ ]:
# ROUGE-1 nasıl kullanılır?

def generate_predictions_and_references(dataset):
  """
    Bu fonksiyon, metni tokenizer.encode ile tokenlaştırır, bu da metni modelin aslında girdi
    olarak aldığı sayısal değerler listesine çıktılar üretir. Bu jetonları içeren modelle, bunları sayı
    listesinden doğal metne geri dönüştürür.
    Ardından referans ve üretilen cevap döndürülür
  """
  predictions = []
  references = []

  for row in dataset:
    inputs = tokenizer.encode(row["instruction"], return_tensors="pt")
    outputs = model.generate(inputs)
    decoded_outputs = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
    # predictions.append(decoded_outputs)
    # references.append(row["response"])
    references += [row["response"]]
    predictions += [decoded_outputs]

  return predictions, references

In [ ]:
references, predictions = generate_predictions_and_references(evaluation_dataset)

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references)

## Efficient Fine-tuning with LoRA


LoRA’yı uygulamak için parameter-efficient fine-tuning (PEFT) kütüphanesi kullanılır. LoRA ayarları için LoraConfig sınıfı tanımlanır:

r: Düşük dereceli matrislerin boyutunu belirler. Küçük r değeri daha az bellek tüketir, büyük r değeri modele daha yakın sonuç verir.

lora_alpha: LoRA tarafından öğrenilen değişimlerin orijinal modele ne kadar yansıtılacağını belirler.

lora_dropout: LoRA katmanlarından rastgele bırakma (dropout) oranını ayarlar.

bias: Bias parametrelerinin eğitilip eğitilmeyeceğini belirler (katman bazlı ya da genel).

task_type: Örneğin Llama için causal_lm olarak ayarlanır.

target_modules: LoRA’nın uygulanacağı büyük dil modeli (LLM) matrislerini belirler.

In [1]:
from peft import LoraConfig

In [ ]:
lora_config = LoraConfig(
    r=12,
    lora_alpha=32 # or scaling factor
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

### r değeri

Standart aralık: 4 ≤ r ≤ 64

En yaygın kullanılan: r = 8

Büyük modeller veya hassas görevler: r = 16 ya da r = 32


Akademik ve Uygulamalı Kaynaklardan Alıntılar:
1. LoRA Orijinal Makalesi (Hu et al., 2021)
LoRA: Low-Rank Adaptation of Large Language Models
📄 arXiv:2106.09685

Çeşitli deneylerde r = 4, 8, 16 değerleri test edilmiştir.

r=8 ve lora_alpha=16 kombinasyonu, performans-maliyet dengesi açısından çok başarılıdır.

2. Hugging Face PEFT dokümantasyonu
https://huggingface.co/docs/peft

r parametresi için doğrudan önerilen bir aralık verilmez, ancak örneklerde genellikle 4, 8 veya 16 kullanılır.

3. Uygulamalı projelerden örnekler (Open LLM Leaderboard, Alpaca-LoRA, etc.)
Alpaca-LoRA: r=8, lora_alpha=16

Baize, Vicuna-LoRA, Koala-LoRA: r=8 yaygın

Çok büyük modellerde (örneğin LLaMA 65B): r=16 veya r=32 tercih edilebiliyor

r Parametresi ve Etkileri:

r Değeri	Bellek Kullanımı	Model Yakınlığı	Kullanım Yaygınlığı
4	Çok düşük	Düşük	Sık (edge device, hızlı test)
8	Düşük	Dengeli	En yaygın
16	Orta	Yüksek	Gelişmiş modeller
32+	Yüksek	Çok yüksek	Büyük modeller, araştırma amaçlı
💡 Notlar:
r değeri arttıkça performans artar ama maliyet (VRAM, zaman) da artar.

LoRA’nın en güçlü yanı, küçük r değerleriyle dahi etkili sonuçlar verebilmesidir.

Uygulamada genellikle lora_alpha = r * 2 şeklinde ayarlanır (örneğin r=8, alpha=16).


### lora_alpha

lora_alpha, LoRA’nın düşük dereceli matrislerinden gelen gradyanların (öğrenme sinyali) orijinal model ağırlıklarına ne kadar etki edeceğini kontrol eden ölçekleme (scaling) katsayısıdır.

r → Ne kadar bilgi taşıyoruz? (boyut)

lora_alpha → Bu bilgiyi ne kadar güçlü uyguluyoruz? (etki gücü)

LoRA’nın temel matematiği şöyle işler:
```
W + ΔW ≈ W + α/r × (A × B)
```

W: Orijinal modelin ağırlığı

A ve B: LoRA'nın öğrenilen düşük-rank matrisleri

α (yani lora_alpha): ölçekleme katsayısı

r: rank değeri

Yani lora_alpha / r oranı, düşük-rank güncellemesinin ne kadar etkili olacağını belirler.

Orijinal LoRA makalesi (Hu et al., 2021):
“We set α such that α / r = 1, 8, or 32 in various experiments. We find α = 16 and r = 8 to be a good balance between performance and stability.”

Yani önerilen ve sık kullanılan kombinasyonlar şunlar:

r = 8, alpha = 16 → α/r = 2 --> en yaygın

r = 4, alpha = 16 → α/r = 4

r = 16, alpha = 32 → α/r = 2


Hugging Face e göre örnek kullanımlar:

Alpaca-LoRA: r=8, lora_alpha=16

Vicuna-LoRA: r=16, lora_alpha=32

LLaMA-LoRA: r=64, lora_alpha=128 veya daha düşük


#### lora_alpha Nasıl Seçilir?

Amaç / Durum	r	lora_alpha	Açıklama
Küçük model, hızlı test	4	8	Daha düşük bellek ve hızlı eğitim
Dengeli kullanım (en yaygın)	8	16	Standarttır, çoğu çalışmada bu kullanılır
Daha hassas uyum, büyük model	16	32 veya 64	Daha fazla bilgi aktarımı
Çok büyük modeller	64	128	Aşırı durumlar, dikkatli kullanılmalı

| **Yüksek `lora_alpha`**               | **Düşük `lora_alpha`**              |
|--------------------------------------|-------------------------------------|
| Daha agresif öğrenme                | Daha yumuşak güncellemeler         |
| Aşırı uyuma (overfitting) riski     | Daha güvenli ama yavaş öğrenme     |
| Büyük modellerde daha etkili        | Küçük modellerde yeterli           |


#### İdeal kombinasyon önerisi
| **r** | **lora_alpha** | **alpha / r** | **Ne zaman tercih edilir?**              |
|------:|----------------:|---------------:|------------------------------------------|
| 4     | 8              | 2             | Hızlı test, düşük bellek kullanımı       |
| 8     | 16             | 2             | En yaygın kullanım, genel denge          |
| 16    | 32             | 2             | Büyük modeller için daha fazla hassasiyet |
| 64    | 128            | 2             | Büyük LLM'ler, araştırma amaçlı kullanım |


### lora_dropout

lora_dropout, LoRA modülündeki düşük-rank matrislerde uygulanacak dropout oranını belirler. Bu, eğitim sırasında bazı nöronların rastgele devre dışı bırakılmasını sağlar.

Dropout, derin öğrenmede aşırı öğrenmeyi (overfitting) önlemek için kullanılan klasik bir teknik olup, LoRA'da da aynı amaçla uygulanır.

#### Yaygın Kullanım Aralıkları
| **Senaryo**                        | **lora_dropout** | **Açıklama**                                               |
|-----------------------------------|------------------|------------------------------------------------------------|
| Aşırı küçük veri kümesi           | 0.1 – 0.2         | Aşırı öğrenmeyi önlemek için daha agresif dropout          |
| Orta büyüklükte, dengeli veri     | 0.05 – 0.1        | En yaygın kullanım aralığı                                 |
| Büyük veri, çok parametreli model | 0.0 – 0.05        | Dropout’a gerek kalmadan genelleme sağlanabilir            |
| Sıfır dropout                     | 0.0               | Maksimum öğrenme, ancak overfitting riski olabilir         |


#### Etkisi
| **Düşük dropout (0.0 – 0.05)**       | **Yüksek dropout (0.1 – 0.2)**            |
|-------------------------------------|-------------------------------------------|
| Daha iyi doğruluk (accuracy)        | Daha yüksek genelleme (generalization)    |
| Aşırı öğrenme riski                 | Öğrenme süresi artabilir                  |
| Büyük modeller için uygun           | Küçük veri setleri için daha güvenli      |


- **Varsayılan öneri**: `lora_dropout = 0.05`
- Küçük veri setlerinde veya çok hızlı öğrenen modellerde: `0.1`
- Çok büyük, zengin veri setleri için (örneğin 1M+ örnek): `0.0 – 0.01`

https://huggingface.co/docs/peft:

"lora_dropout" is a dropout probability to apply to the LoRA layers during training. Defaults to 0.0, but values like 0.05 – 0.1 are common for regularization.


### bias
bias parametresi, orijinal modeldeki bias (sapma) terimlerinin ince ayar sırasında eğitilip eğitilmeyeceğini belirler.

Normalde LoRA sadece weight matrislerini hedef alır. Ama bazı modellerde bias terimleri de önemli olabilir. Bu parametre, bu bias değerlerinin:

Eğitilip eğitilmeyeceğini

Hangi seviyede eğitileceğini (global mi, katman bazlı mı)

belirlememizi sağlar.

Kullanım Değerleri ve Anlamları
```
bias="none"         # Varsayılan. Bias parametreleri eğitilmez. Sadece LoRA ağırlıkları eğitilir, daha hafif model
bias="all"          # Tüm bias terimleri eğitilir. Tüm modeldeki bias'lar eğitilir. Daha fazla kontrol, ama daha fazla parametre
bias="lora_only"    # Sadece LoRA modülünün içindeki bias'lar eğitilir. LoRA ile ilişkili bias'lar da eğitilir. Dengeli bir seçimdir.
```

- **Hızlı ve minimum kaynak kullanımı** için: `bias="none"` (varsayılan)
- **LoRA modülünde daha hassas öğrenme** isteniyorsa: `bias="lora_only"`
- **Tüm modelde detaylı ince ayar** gerekiyorsa (özellikle küçük modellerde): `bias="all"`


| **Bias Ayarı**    | **Anlamı**                                                                 |
|-------------------|-----------------------------------------------------------------------------|
| `"none"`          | Hiçbir bias parametresi eğitilmez (varsayılan).                            |
| `"lora_only"`     | Sadece LoRA katmanlarının içindeki bias parametreleri eğitilir.            |
| `"all"`           | Modeldeki tüm bias parametreleri eğitilir. Daha fazla hesaplama gerektirir. |


| **Kullanım Durumu**                          | **Tercih Edilen Bias Ayarı** |
|---------------------------------------------|-------------------------------|
| Minimum bellek, hızlı eğitim                | `"none"`                     |
| LoRA'nın kendi içsel bias öğrenimini istersen | `"lora_only"`                |
| Daha ince ayar, detaylı müdahale gereken modeller | `"all"`                  |







### task_type

task_type, LoRA’nın uygulanacağı model mimarisini ve görev türünü belirtir. Bu, LoRA’nın hangi katmanlara entegre edileceğini, hangi ayarlarla çalışacağını belirler.

Her model ve görev türü için LoRA’nın etkili çalışabilmesi adına farklı hedef modüller gerekebilir. Bu yüzden task_type parametresi, modelin yapısına özel doğru kurulum yapılabilmesi için gereklidir.


LoRA bir Transformer modeline entegre edilirken, mimariye göre doğru layer’lara bağlanmalıdır. task_type parametresi PEFT kütüphanesine şu sinyali verir:

“Ben causal language model mi eğitiyorum, yoksa bir sınıflandırma modeli mi?“

Bu bilgiye göre hangi katmanlara LoRA uygulanacağını otomatik olarak ayarlar.

 task_type Değerleri ve Açıklamaları

| **task_type**           | **Açıklama**                                                            |
|-------------------------|-------------------------------------------------------------------------|
| "CAUSAL_LM"             | Metin üretimi modelleri (LLaMA, GPT, Falcon vb.)                        |
| "SEQ_CLS"               | Sıralama sınıflandırması (BERT ile duygu analizi)                      |
| "TOKEN_CLS"             | Token düzeyinde sınıflandırma (adlandırılmış varlık tanıma)             |
| "SEQ_2_SEQ_LM"          | Encoder-decoder tabanlı modeller (T5, BART, çeviri vb.)                 |
| "QUESTION_ANSWERING"    | Soru-cevap görevleri (bert-base-uncased + SQuAD gibi)                  |


Hatalardan Kaçınmak İçin
Modelin mimarisi ile uyumlu bir task_type belirtmezsen, LoRA katmanları yanlış yerlere uygulanabilir veya hiç uygulanmayabilir.

Hugging Face model sayfasındaki model_class bilgisi genellikle hangi task_type’ın seçilmesi gerektiğini gösterir.

| **Model Türü / Kullanım Senaryosu**          | **Uygun `task_type`** |
|---------------------------------------------|------------------------|
| GPT-2, LLaMA, Falcon                         | "CAUSAL_LM"            |
| T5, BART, mT5                                | "SEQ_2_SEQ_LM"         |
| BERT ile metin sınıflandırma                | "SEQ_CLS"              |
| BERT ile kelime seviyesinde sınıflandırma   | "TOKEN_CLS"            |
| Soru-Cevap (BERT + SQuAD gibi datasetler)    | "QUESTION_ANSWERING"   |


### target_module

target_modules, LoRA’nın hangi nöral ağ katmanlarına (modüllerine) entegre edileceğini belirtir. Bu, LoRA'nın ağırlık güncellemelerini hangi katmanlara uygulayacağını belirleyen en önemli hiperparametrelerden biridir.

LoRA mimarisinde yalnızca belirli katmanlara "düşük-rank güncelleme" (low-rank update) uygulanır. Bu katmanları manuel olarak belirtmek, özelleştirilmiş ve verimli bir ince ayar sağlar.

1. LLaMA / GPT Tarzı Modeller (CAUSAL_LM) **kalın metin**

`target_modules = ["q_proj", "v_proj"]`

Q ve V projeksiyonları, transformer'da self-attention mekanizmasının kalbidir.

Bu iki modül yeterli LoRA etkileşimi sağlar ve oldukça yaygındır.


2. BERT Tarzı Modeller (SEQ_CLS / TOKEN_CLS **kalın metin**)

`target_modules = ["query", "value"]`

BERT’in attention başlıkları genelde query, key, value olarak adlandırılır.

query ve value yeterli olur, key genellikle eklenmez.

**3. T5 / BART Gibi Seq2Seq Modeller**

`target_modules = ["q", "v", "k", "o"]`

Bu modellerde farklı adlandırmalar olabilir (q, k, v, o) — modellerin implementation'ına dikkat etmek gerekir.

| **Model Türü**     | **task_type**   | **Önerilen `target_modules`**     |
|--------------------|-----------------|-----------------------------------|
| LLaMA / GPT        | "CAUSAL_LM"     | ["q_proj", "v_proj"]              |
| BERT (sınıflama)   | "SEQ_CLS"       | ["query", "value"]                |
| BERT (etiketleme)  | "TOKEN_CLS"     | ["query", "value"]                |
| T5, BART (çeviri)  | "SEQ_2_SEQ_LM"  | ["q", "v"]                        |


| **Hedef**                                   | **target_modules Stratejisi**      |
|--------------------------------------------|------------------------------------|
| En yaygın LoRA kullanımı (standart setup)  | ["q_proj", "v_proj"]               |
| Daha agresif müdahale                      | ["q_proj", "v_proj", "k_proj"]     |
| Daha hafif, hızlı eğitim                   | ["v_proj"]                         |
| BERT tabanlı ince ayar                     | ["query", "value"]                 |


In [ ]:
# Using LoRA Adapters
# 🎯 Bu örnekte, müşteri hizmetleri alanında kullanılan bir sohbet botu için
# Maykeye/TinyLlama-v0 modeli üzerinde LoRA (Low-Rank Adaptation) kullanılarak
# verimli bir ince ayar süreci uygulanmaktadır. Eğitim betiği hazırdır,
# amaç yalnızca LoRA yapılandırmasını entegre etmektir.
#
# 🔧 Uygulama Adımları:
# - `peft` kütüphanesinden LoRA yapılandırmasını içe aktarın.
# - Varsayılan değerlerle bir `lora_config` oluşturun.
# - Bu yapılandırmayı `SFTTrainer` sınıfına dahil ederek eğitimi başlatın.

In [ ]:
# Import LoRA configuration class
from peft import LoraConfig

In [ ]:
lora_config = LoraConfig(
    r=12,
    lora_alpha=8,
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
    bias="none",
    target_modules=['q_proj', 'v_proj']
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_arguments,
    # Pass the lora_config to trainer
    peft_config=lora_config
)

In [ ]:
# 🧪 Bu çalışmada, sınırlı donanım kaynaklarıyla dil modeli ince ayarı yapılmaktadır.
# Geleneksel fine-tuning yerine daha hafif bir yöntem olan LoRA kullanılmıştır.
# Model olarak TinyLlama tercih edilmiş, örnek veriler bitext benzeri bir yapıdadır.
#
# Eğitim kodları büyük ölçüde hazırdır. Bu hücrede yapılması gereken şey:
# - LoRA yapılandırmasında rank değerini 2 olarak ayarlamak,
# - lora_alpha'yı bu değerin 2 katı (yani 4) yapmak,
# - ve görev türünü Llama tipi modellerle uyumlu olacak şekilde "CAUSAL_LM" olarak belirtmektir.
#
# Gerekli değişkenler (model, tokenizer, dataset, training_arguments) önceden tanımlanmıştır.


In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
peft_config = LoraConfig(
    # Set rank parameter
    r=2,
    # Set scaling factor (2 * r)
    lora_alpha=4,
    # Set the type of task for LLaMA-style models
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
    bias="none",
    target_modules=['q_proj', 'v_proj']
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=250,
    dataset_text_field='conversation',
    tokenizer=tokenizer,
    peft_config=peft_config
)

# LoRA ve Attention Mekanizmasında q, k, v Yapıları

## 1. q, k ve v Nedir?

`q`, `k` ve `v` terimleri, transformer modellerinde kullanılan **self-attention** mekanizmasının temel bileşenleridir.  
Her biri, giriş verisinin farklı alt uzaylara projeksiyonudur. Bu projeksiyonlar sayesinde model, hangi bilgilere dikkat edeceğini ve ne aktaracağını belirler.

- **q (query)**: Sorgu vektörüdür. Modelin hangi bilgiye dikkat etmesi gerektiğini temsil eder.
- **k (key)**: Anahtar vektörüdür. Her kelimenin tanımlayıcı bilgisini taşır, sorguya cevap oluşturur.
- **v (value)**: Değer vektörüdür. Asıl içeriği taşıyan bileşendir.

Bu projeksiyonlar şu şekilde elde edilir:

Q = X · W_q

K = X · W_k

V = X · W_v


Attention(Q, K, V) = softmax(QKᵀ / √d_k) · V




Bu formül, her kelimenin diğer kelimelere ne kadar dikkat etmesi gerektiğini bulur.  
- `QKᵀ`: İlgililik skorlarını üretir (query-key benzerliği)
- `V`: Asıl bilgi buradan aktarılır

---

## 3. LoRA Bu Yapılarla Nasıl İlgili?

LoRA (Low-Rank Adaptation), büyük dil modellerine **düşük parametreli adaptasyon katmanları** ekleyerek verimli ince ayar (fine-tuning) yapmayı sağlar.  
Bu adaptasyon katmanları genellikle `q_proj` ve `v_proj` gibi modüllere uygulanır.

### Neden `q_proj` ve `v_proj`?

- **q_proj (Query Projeksiyonu)**: Dikkat yönlendirmesini doğrudan etkiler.
- **v_proj (Value Projeksiyonu)**: Modelin içerik taşımasını doğrudan etkiler.
- Bu iki modül, modelin çıktı üretme kalitesinde belirleyicidir.

### Neden `k_proj` Genellikle Dahil Edilmez?

- `k_proj` (Key projeksiyonu), sorgularla eşleşme kurallarını belirler.
- Bu yapı sabit kalırsa model daha istikrarlı davranır.
- Fazla müdahale, dikkat mekanizmasının dengesini bozabilir.

---

## 4. Yaygın target_modules Ayarları

Aşağıda farklı model türlerine göre önerilen `target_modules` listesi verilmiştir:

| Model Türü         | task_type       | target_modules             |
|--------------------|-----------------|----------------------------|
| LLaMA, GPT         | "CAUSAL_LM"     | ['q_proj', 'v_proj']       |
| BERT (sınıflama)   | "SEQ_CLS"       | ['query', 'value']         |
| BERT (etiketleme)  | "TOKEN_CLS"     | ['query', 'value']         |
| T5, BART           | "SEQ_2_SEQ_LM"  | ['q', 'v']                 |

---

## 5. Özet

- `q`, `k` ve `v`, giriş vektörlerinin öğrenilmiş matrislerle farklı görevlere projekte edilmiş (iz düşürülmüş) halleridir.
- Transformer'daki dikkat (attention) işlemleri bu projeksiyonlar üzerinden çalışır.
- LoRA, modelin tüm ağırlıklarını değil, yalnızca `q_proj` ve `v_proj` gibi hassas katmanları güncelleyerek verimli adaptasyon sağlar.
- Bu sayede daha az parametreyle, daha hızlı ve daha ekonomik fine-tuning mümkündür.

---

## Kaynaklar

- Vaswani et al., 2017. [Attention is All You Need](https://arxiv.org/abs/1706.03762)
- Hu et al., 2021. [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- Hugging Face PEFT Documentation: https://huggingface.co/docs/peft



# Kuantizasyon ile daha küçük model, daha hızlı çıkarım



---
Model kuantizasyonu, modelleri daha düşük hassasiyetli bir formata dönüştürerek bellek kullanımını azaltır ve çıkarım hızını artırır. Parametreler ve aktivasyonlar daha az bit kullanır; 32 bitlik floatlardan 8 bitlik veya 4 bitlik tam sayılara geçilir, bu da bellek kullanımını azaltır. Kuantizasyon uygulandığında, modelin performansı üzerindeki etkisini en aza indirmek için eğitim süreci ayarlanabilir.

Kuantize edilmiş bir model doğrudan ince ayar yapılamaz çünkü ağırlıklar ayrıklaştırılmıştır. Bu sorunu aşmak için daha önce öğrendiğimiz LoRA adaptasyonunu kullanabiliriz. SFTTrainer’ı kullanarak, bitsandbytesconfig ile yüklenen kuantize edilmiş modeli aktarabiliriz ve LoRA yapılandırmasının bir örneğini peft_config olarak peft kütüphanesi aracılığıyla aktarabiliriz. Ardından, modeli her zamanki gibi LoRA adaptasyonu ile trainer.train komutu ile eğitebiliriz.

| Teknik                           | Ne Yapar?                                                                      | Amaç                                                           |
|----------------------------------|--------------------------------------------------------------------------------|----------------------------------------------------------------|
| **Ağırlık Kuantizasyonu**        | Ağırlıkları daha düşük bit’li sayılarla temsil eder                            | Modelin boyutunu ve belleğini azaltmak                        |
| **Aktivasyon Kuantizasyonu**     | Ara aktivasyonları daha düşük bit ile hesaplar                                 | Anlık RAM kullanımını ve işlem gücünü azaltmak                |
| **PTQ (Post-Training Quantization)** | Eğitimden sonra kuantizasyon uygular                                      | Eğitim verisi gerektirmeden model verimliliğini artırmak      |
| **LoRA**                         | Tüm ağırlıkları değil, sadece kritik parçaları günceller                        | Az parametreyle fine-tuning yapabilmek                        |
| **Distillation (Bilgi Yoğunlaştırma)** | Büyük modelin bilgisini küçük modele aktarır                            | Daha küçük modelle benzer doğruluk sağlamak                   |


In [1]:
# Loading model with quantization
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

In [ ]:
bnb_config = BitsAndBytesConfig(
    # set precision (load_in_4_bit, load_in_8_bit)
    load_in_4bit=True,
    # set quantization type ('fp4' or 4-bit float, 'nf4' or normalized 4-bit float)
    bnb_4bit_quant_type="nf4",
    # set compute dtype (compute precision) (float16, float32, bfloat16)
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/Llama3-ChatQA-1.5-8B",
    quantization_config=bnb_config,
)

In [ ]:
# Kuantize edilmiş modeli kullanmak:
promptstr = """System: You are a helpful chatbot who answers questions about planets.
User: Explain the history of Mars.
Assistant: """
inputs = tokenize.encode(promptstr, return_tensors="pt")
outputs = model.generate(inputs, max_length=200)
decoded_outputs = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
print(decoded_outputs)

In [ ]:
# Normalde kuantize edilmiş model üzerine fine-tune işlemi yapamayız.
# Ancak bu kısıtlamayı Lora ile aşarız
trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    train_dataset=ds,
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=250,
    dataset_text_field='conversation',
)

trainer.train()

In [ ]:
# Chatbot için kullanılan modelin GPU bellek kullanımını performanstan çok fazla ödün vermeden azaltmak
# Modeli 8-bit kuantizasyonuyla yüklenecek
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
model_name = "Maykeye/TinyLLama-v0"

bnb_config = BitsAndBytesConfig(
    # Set 8-bit loading
    load_in_8_bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # Set quantization parameters
    to load quantized model
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Maykeye/TinyLLama-v0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    quant_type="nf4",                     # Normalized 4-bit quantization
    bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for faster inference
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)